## Python3.3 yield from 语法

- chain 可以将迭代的类型连接起来

In [2]:
from itertools import chain

my_list = [1, 2 ,3]
my_dict = {
    "bobby1": "http://projectsedu.com",
    "bobby2": "http://www.imooc.com"
}

for value in chain(my_list, my_dict, range(5, 10)):
    print(value)

1
2
3
bobby1
bobby2
5
6
7
8
9


- 实现自己的 chain

In [3]:
my_list = [1, 2 ,3]
my_dict = {
    "bobby1": "http://projectsedu.com",
    "bobby2": "http://www.imooc.com"
}

def my_chain(*args, **kwargs):
    for my_iterable in args:
        for value in my_iterable:
            yield value

for value in my_chain(my_list, my_dict, range(5, 10)):
    print(value)

1
2
3
bobby1
bobby2
5
6
7
8
9


- yield from 的用法

In [4]:
my_list = [1, 2 ,3]
my_dict = {
    "bobby1": "http://projectsedu.com",
    "bobby2": "http://www.imooc.com"
}

# yield from iterable
def my_chain(*args, **kwargs):
    for my_iterable in args:
        yield from my_iterable

for value in my_chain(my_list, my_dict, range(5, 10)):
    print(value)

1
2
3
bobby1
bobby2
5
6
7
8
9


- yield from 功能演示2

In [5]:
def g1(iterable):
    yield iterable
    
def g2(iterable):
    yield from iterable

for value in g1(range(10)):
    print(value)

print("---------------------------")
    
for value in g2(range(10)):
    print(value)

range(0, 10)
---------------------------
0
1
2
3
4
5
6
7
8
9


- yield from 功能演示3
    - 会直接建立 子生成器 与 调用方的 双向 chan

In [9]:
# 子生成器
def gen(n):
    yield from range(n)

# 委托生成器
def g1(gen):
    yield from gen

# 调用方
def main():
    g = g1(gen(4))
    g.send(None)
    for value in g:
        print(value)

main()

1
2
3


- yield_from 综合例子

In [10]:
final_result = {}

def sales_sum(pro_name):
    total = 0
    nums = []
    while True:
        x = yield
        print(pro_name+" 销量：", x)
        if not x:
            break
        total += x
        nums.append(x)
    return total, nums

def delegationGen(key):
    while True:
        final_result[key] = yield from sales_sum(key)
        print(key+"销量统计完成！！！")
        
def main():
    data_sets = {
        "面膜": [1200, 1500, 2100],
        "手机": [28, 55, 98, 108],
        "大衣": [280, 560, 78, 90]
    }
    for key, data_set in data_sets.items():
        print("start key:", key)
        delegate = delegationGen(key)
        delegate.send(None)
        for value in data_set:
            delegate.send(value)
        delegate.send(None)
    print("final_result:", final_result)
    
if __name__ == "__main__":
    main()

start key: 面膜
面膜 销量： 1200
面膜 销量： 1500
面膜 销量： 2100
面膜 销量： None
面膜销量统计完成！！！
start key: 手机
手机 销量： 28
手机 销量： 55
手机 销量： 98
手机 销量： 108
手机 销量： None
手机销量统计完成！！！
start key: 大衣
大衣 销量： 280
大衣 销量： 560
大衣 销量： 78
大衣 销量： 90
大衣 销量： None
大衣销量统计完成！！！
final_result: {'面膜': (4800, [1200, 1500, 2100]), '手机': (289, [28, 55, 98, 108]), '大衣': (1008, [280, 560, 78, 90])}


package main

import (
	"fmt"
)

func salesSum(proName string, ch chan int) (int, []int) {
	total := 0
	nums := []int{}
	for x := range ch {
		fmt.Printf("%s 销量： %d\n", proName, x)
		total += x
		nums = append(nums, x)
	}
	return total, nums
}

func delegationGen(key string, data []int, resultCh chan map[string][2]int) {
	finalResult := make(map[string][2]int)
	ch := make(chan int)

	// Goroutine to process salesSum
	go func() {
		total, nums := salesSum(key, ch)
		finalResult[key] = [2]int{total, len(nums)} // Store total sales and count of sales
		fmt.Println(key + "销量统计完成！！！")
		// Send the final result back to the main function via resultCh
		resultCh <- finalResult
	}()

	// Sending data to the salesSum goroutine through the channel
	for _, value := range data {
		ch <- value
	}

	// Close the channel when done sending data
	close(ch)
}

func main() {
	dataSets := map[string][]int{
		"面膜": {1200, 1500, 2100},
		"手机": {28, 55, 98, 108},
		"大衣": {280, 560, 78, 90},
	}

	finalResult := make(map[string][2]int)
	resultCh := make(chan map[string][2]int)

	// Start processing each product data in a goroutine
	for key, dataSet := range dataSets {
		fmt.Println("start key:", key)
		go delegationGen(key, dataSet, resultCh)
	}

	// Collect results from the channels
	for range dataSets {
		partialResult := <-resultCh
		for k, v := range partialResult {
			finalResult[k] = v
		}
	}

	// Print final result after all goroutines have completed
	fmt.Printf("final_result: %v \r\n", finalResult)
}

start key: 面膜
start key: 手机       
面膜 销量： 1200      
面膜 销量： 1500      
面膜 销量： 2100      
面膜销量统计完成！！！
手机 销量： 28        
start key: 大衣       
手机 销量： 55
手机 销量： 98
手机 销量： 108
手机销量统计完成！！！
大衣 销量： 280
大衣 销量： 560
大衣 销量： 78
大衣 销量： 90
大衣销量统计完成！！！
final_result: map[大衣:[1008 4] 手机:[289 4] 面膜:[4800 3]]